In [17]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [18]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor(),
)

In [19]:
batch_size = 100
train_dataloader = DataLoader(training_data, batch_size=100,shuffle=True)

In [65]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 5*5*16)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [66]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [67]:
tb = SummaryWriter()

network = NeuralNetwork()
images, labels = next(iter(train_dataloader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(10):
    total_loss, total_correct= 0,0
    for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct+= (pred.argmax(1) == y).type(torch.float).sum().item()
    tb.add_scalar('Loss',total_loss,epoch)
    tb.add_scalar('Number Correct',total_correct,epoch)
    tb.add_scalar('Accuracy',total_correct/len(train_dataloader.dataset),epoch)
    
    tb.add_scalars('Loss vs Correct', {'Loss':total_loss,
                                                            'Correct':total_correct }, epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias,epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight,epoch)
    
    print("epoch:", epoch+1, "total correct:", total_correct, "loss:", total_loss)

epoch: 1 total correct: 47670.0 loss: 326.36219312250614
epoch: 2 total correct: 51985.0 loss: 215.94076070189476
epoch: 3 total correct: 52774.0 loss: 195.52665321528912
epoch: 4 total correct: 53121.0 loss: 184.171044588089
epoch: 5 total correct: 53563.0 loss: 175.72666830569506
epoch: 6 total correct: 53839.0 loss: 167.72783072292805
epoch: 7 total correct: 54085.0 loss: 160.86262968927622
epoch: 8 total correct: 54159.0 loss: 158.8448846936226
epoch: 9 total correct: 54219.0 loss: 157.03925355523825
epoch: 10 total correct: 54327.0 loss: 155.1128320991993


In [ ]:
tb.close()